# Sprint Retrospective Analysis

## Objective
Analyze historical sprint data to identify areas for improvement and enhance future sprint planning.

## Instructions
Follow the steps provided in this notebook to load the data, preprocess it, perform exploratory data analysis, train a predictive model, and draw insights for sprint retrospectives.


In [ ]:
# Install necessary libraries
!pip install -r requirements.txt


In [ ]:
# Step 1: Load the Dataset
import pandas as pd

# Load dataset
data = pd.read_csv('sprint_data.csv')

# Explore dataset
print("First 5 rows of the dataset:")
print(data.head())
print("\nSummary statistics of the dataset:")
print(data.describe())


In [ ]:
# Step 2: Data Preprocessing

# Handle missing values (if any)
data = data.dropna(subset=['estimated_hours', 'actual_hours', 'completion_status'])

# Select relevant columns
relevant_columns = ['sprint_id', 'team_member', 'task_id', 'task_description', 'estimated_hours', 'actual_hours', 'completion_status']
data = data[relevant_columns]

# Create new feature for time difference
data['time_diff'] = data['actual_hours'] - data['estimated_hours']

# Display the first few rows of the preprocessed dataset
print("First 5 rows of the preprocessed dataset:")
print(data.head())


In [ ]:
# Step 3: Exploratory Data Analysis

import matplotlib.pyplot as plt
import seaborn as sns

# Plot task completion rates
completion_rate = data['completion_status'].value_counts(normalize=True) * 100
plt.figure(figsize=(8, 5))
sns.barplot(x=completion_rate.index, y=completion_rate.values)
plt.title('Task Completion Rates')
plt.xlabel('Completion Status')
plt.ylabel('Percentage')
plt.show()

# Plot estimated vs actual time
plt.figure(figsize=(10, 6))
sns.scatterplot(x='estimated_hours', y='actual_hours', data=data, hue='completion_status')
plt.title('Estimated vs Actual Time for Tasks')
plt.xlabel('Estimated Hours')
plt.ylabel('Actual Hours')
plt.show()

# Plot time difference by sprint
plt.figure(figsize=(12, 6))
sns.boxplot(x='sprint_id', y='time_diff', data=data)
plt.title('Time Difference by Sprint')
plt.xlabel('Sprint ID')
plt.ylabel('Time Difference (Actual - Estimated Hours)')
plt.axhline(0, color='r', linestyle='--')
plt.show()

# Plot average time difference by team member
team_member_perf = data.groupby('team_member')['time_diff'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='team_member', y='time_diff', data=team_member_perf)
plt.title('Average Time Difference by Team Member')
plt.xlabel('Team Member')
plt.ylabel('Average Time Difference (Actual - Estimated Hours)')
plt.axhline(0, color='r', linestyle='--')
plt.show()


In [ ]:
# Step 4: Model Training and Evaluation

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Encode categorical variables
team_member_encoder = LabelEncoder()
task_description_encoder = LabelEncoder()

data['team_member_encoded'] = team_member_encoder.fit_transform(data['team_member'])
data['task_description_encoded'] = task_description_encoder.fit_transform(data['task_description'])

# Standardize numerical features
scaler = StandardScaler()
data[['estimated_hours', 'actual_hours', 'time_diff']] = scaler.fit_transform(data[['estimated_hours', 'actual_hours', 'time_diff']])

# Prepare data for model training
X = data[['team_member_encoded', 'task_description_encoded', 'estimated_hours']]
y = data['actual_hours']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate model performance
print("Model performance on the test set:")
print(f'R^2 Score: {model.score(X_test, y_test)}')


In [ ]:
# Step 5: Predictive Analysis for Sprint 6

# Prepare data for sprint 6
sprint_6 = data[data['sprint_id'] == 6].copy()
sprint_6['team_member_encoded'] = team_member_encoder.transform(sprint_6['team_member'])
sprint_6['task_description_encoded'] = task_description_encoder.transform(sprint_6['task_description'])

# Predict estimated_hours and actual_hours for sprint 6
sprint_6['estimated_hours'] = model.predict(sprint_6[['team_member_encoded', 'task_description_encoded', 'estimated_hours']])
sprint_6['actual_hours'] = model.predict(sprint_6[['team_member_encoded', 'task_description_encoded', 'estimated_hours']])

# Decode the categorical variables
sprint_6['team_member'] = team_member_encoder.inverse_transform(sprint_6['team_member_encoded'])
sprint_6['task_description'] = task_description_encoder.inverse_transform(sprint_6['task_description_encoded'])

# Display predictions for sprint 6
print("Predictions for sprint 6:")
print(sprint_6[['team_member', 'task_id', 'task_description', 'estimated_hours', 'actual_hours']])


## Conclusion

In this analysis, we loaded and preprocessed sprint data, conducted exploratory data analysis to visualize task completion rates and time discrepancies, trained a machine learning model to predict sprint outcomes, and generated actionable insights based on the model's predictions. By leveraging these insights, teams can improve sprint planning, allocate resources more effectively, and enhance overall productivity.

Further analysis can include trend analysis over multiple sprints and correlation analysis between different variables to gain deeper insights into the factors influencing sprint outcomes.
